In [1]:
import findspark
findspark.init('/home/navin/spark-2.4.6-bin-hadoop2.7')

In [2]:
from datetime import datetime

In [3]:
print(datetime.now().strftime("%Y%m%d%H%M%S"))

20240531175659


In [4]:
# word count program
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("WordCount") \
    .getOrCreate()

In [5]:
# Read the text file into an RDD
text_file = spark.sparkContext.textFile("file:///home/navin/Downloads/wordcount.txt")

# Perform the WordCount
counts = text_file.flatMap(lambda line: line.split(" ")) \
                  .map(lambda word: (word, 1)) \
                  .reduceByKey(lambda a, b: a + b)

In [6]:
# flatmap_op=text_file.flatMap(lambda line: line.split(" "))
# text_file.flatMap(lambda line: line.split(" ")).collect()
# text_file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).collect()
text_file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 2)).reduceByKey(lambda a, b: a + b).collect()
# res=flatmap_op.collect()
# print(res)

[('world!', 2), ('home', 2), ('hello', 4), ('welcome', 4)]

In [20]:
type(counts)

pyspark.rdd.PipelinedRDD

In [21]:
# counts.saveAsTextFile("file:///home/navin/Downloads/output")

In [22]:
# Convert RDD into DataFrame
counts_df = counts.toDF(["word", "count"])

In [23]:
# Coalesce to a single partition
counts_single_partition = counts_df.coalesce(1)

timestamp=datetime.now().strftime("%Y%m%d%H%M%S")
outputFilePath=f"file:///home/navin/Downloads/output_{timestamp}"

# Save the result to HDFS
counts_single_partition.write.mode("overwrite").csv(outputFilePath)


# why spark.stop()?
Resource Management:
- Spark applications run in a distributed environment, often using multiple nodes in a cluster. When a Spark context is created, it allocates resources (like memory and CPU) across these nodes. Stopping the context releases these resources, making them available for other applications or processes.

In [7]:
# Stop the SparkSession
spark.stop()

#  spark - submit -PY FILE
$SPARK_HOME/bin/spark-submit --master spark://<master-node>:7077 /path/to/wordcount.py

bin/spark-submit --master spark://navin-VM:7077 --num-executors 1 /home/navin/Documents/wordcount_program.py